# Akeneo Data Cache

For easier access to the data, a cache was implemented. That is a class, that queries all Akeneo PIM endpoints and stores the result data in json files in a cache directory. The cache class now also offers methods to load the data from these files again and parse them them to more useful python data classes. This much fast than querying the Akeneo Api every time data ist needed.

In [1]:
import shutil

from IPython.display import display
import pandas as pd

from src import akeneo, config

Prepare directory to store the parsed data classes:

In [2]:
csv_dir = config.dir_data / "akeneo-api-cache"
shutil.rmtree(csv_dir, ignore_errors=True)
csv_dir.mkdir(parents=True, exist_ok=True)

In [3]:
def print_and_safe_data(name: str, data: list):
    df = pd.DataFrame(data)

    print(f"--- {name} --------------------------")
    display(df)
    
    df.to_csv(csv_dir / f"{name}.csv", index=False)

Init the cache from environment variables (see `.env` file):

In [2]:
cache = akeneo.create_cache_from_env()

Fetch the data from Akeneo Api:

In [3]:
cache.fetch_data_from_api()

Take look at some of the parsed data from the json files:

In [6]:
pd.DataFrame(cache.currencies)

,code,enabled
0,ADP,False
1,AED,False
2,AFA,False
3,AFN,False
4,ALK,False
...,...,...
289,ZRN,False
290,ZRZ,False
291,ZWD,False
292,ZWL,False


In [7]:
result = [
    {**cat.__dict__, "label_en": cat.get_label("en_US"), "label_de": cat.get_label("de_DE")}
    for cat in cache.categories
]
pd.DataFrame(result)

,code,labels,parent,label_en,label_de
0,master,"{'en_US': 'Master catalog', 'de_DE': 'Hauptkat...",None,Master catalog,Hauptkatalog
1,tvs_projectors,"{'en_US': 'TVs and projectors', 'de_DE': 'TVs ...",master,TVs and projectors,TVs und projectoren
2,pc_monitors,"{'en_US': 'PC Monitors', 'de_DE': 'PC Monitore...",tvs_projectors,PC Monitors,PC Monitoren
3,led_tvs,"{'en_US': 'LED TVs', 'de_DE': 'TVs LED', 'fr_F...",tvs_projectors,LED TVs,TVs LED
4,cameras,"{'en_US': 'Cameras', 'de_DE': 'Cameras', 'fr_F...",master,Cameras,Cameras
...,...,...,...,...,...
163,suppliers,"{'en_US': 'Suppliers', 'de_DE': 'Suppliers', '...",None,Suppliers,Suppliers
164,supplier_mongo,"{'en_US': 'Mongo', 'de_DE': 'Mongo', 'fr_FR': ...",suppliers,Mongo,Mongo
165,supplier_zaro,"{'en_US': 'Zaro', 'de_DE': 'Zaro', 'fr_FR': 'Z...",suppliers,Zaro,Zaro
166,supplier_the_tootles,"{'en_US': 'The Tootles', 'de_DE': 'The Tootles...",suppliers,The Tootles,The Tootles


Print and store to csv all data structures available from the cache:

In [8]:
data_types = [
    ("attribute-groups", cache.attribute_groups),
    ("attributes", cache.attributes),
    ("categories", cache.categories),
    ("channels", cache.channels),
    ("currencies", cache.currencies),
    ("families", cache.families),
    ("locales", cache.locales),
    ("measurements", cache.measurements),
    ("products", cache.products),
]
for name, data in data_types:
    print_and_safe_data(name, data)

--- attribute-groups --------------------------


,code,labels,attributes,sort_order
0,color,"{'en_US': 'Color', 'fr_FR': 'Couleur', 'de_DE'...","[main_color, secondary_color]",8
1,design,"{'en_US': 'Design', 'fr_FR': 'Style', 'de_DE':...",[tshirt_style],3
2,ecommerce,"{'en_US': 'Ecommerce', 'fr_FR': 'Ecommerce', '...","[meta_title, meta_description, keywords]",6
3,erp,"{'en_US': 'ERP', 'fr_FR': 'ERP', 'de_DE': 'ERP'}","[sku, price, erp_name, ean, supplier]",1
4,manufacturing,"{'en_US': 'Manufacturing', 'fr_FR': 'Fabricati...","[container_material, tshirt_materials]",6
5,marketing,"{'en_US': 'Marketing', 'fr_FR': 'Marketing', '...","[name, description, response_time, release_dat...",0
6,medias,"{'en_US': 'Media', 'fr_FR': 'Media', 'de_DE': ...","[picture, pdf_description, image, notice, vari...",9
7,other,"{'en_US': 'Other', 'fr_FR': 'Autre', 'de_DE': ...",[],10
8,product,"{'en_US': 'Product', 'fr_FR': 'Produit', 'de_D...","[composition, wash_temperature, care_instructi...",4
9,size,"{'en_US': 'Size', 'fr_FR': 'Taille', 'de_DE': ...",[clothing_size],8


--- attributes --------------------------


,code,labels,type,localizable,scopable,unique,group,group_labels,sort_order,allowed_extensions,...,max_file_size,metric_family,minimum_input_length,negative_allowed,number_min,number_max,reference_data_name,validation_rule,validation_regexp,wysiwyg_enabled
0,auto_exposure,"{'de_DE': 'Auto exposure', 'en_US': 'Auto expo...",AttributeType.BOOL,False,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",39,[],...,None,None,None,None,None,None,None,None,None,None
1,auto_focus_assist_beam,"{'de_DE': 'Auto focus beam', 'en_US': 'Auto fo...",AttributeType.BOOL,False,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",34,[],...,None,None,None,None,None,None,None,None,None,None
2,auto_focus_lock,"{'de_DE': 'Auto focus lock', 'en_US': 'Auto fo...",AttributeType.BOOL,False,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",33,[],...,None,None,None,None,None,None,None,None,None,None
3,auto_focus_modes,"{'de_DE': 'Auto focus modes', 'en_US': 'Auto f...",AttributeType.TEXT,True,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",31,[],...,None,None,None,None,None,None,None,None,None,None
4,auto_focus_points,"{'de_DE': 'Auto focus points', 'en_US': 'Auto ...",AttributeType.NUMBER,False,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",32,[],...,None,None,None,False,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,variation_image,"{'de_DE': 'Variant bild', 'en_US': 'Variant pi...",AttributeType.IMAGE,False,False,False,medias,"{'en_US': 'Media', 'fr_FR': 'Media', 'de_DE': ...",20,"[png, jpeg, jpg]",...,None,None,None,None,None,None,None,None,None,None
73,variation_name,"{'de_DE': 'Variant Name', 'en_US': 'Variant Na...",AttributeType.TEXT,True,False,False,marketing,"{'en_US': 'Marketing', 'fr_FR': 'Marketing', '...",4,[],...,None,None,None,None,None,None,None,None,None,None
74,viewing_area,"{'de_DE': 'Effektive Bildfläche', 'en_US': 'Ef...",AttributeType.METRIC,False,False,False,technical,"{'en_US': 'Technical', 'fr_FR': 'Technique', '...",20,[],...,None,Length,None,False,None,None,None,None,None,None
75,wash_temperature,"{'de_DE': 'Wash temperature', 'en_US': 'Wash t...",AttributeType.SELECT_SINGLE,False,False,False,product,"{'en_US': 'Product', 'fr_FR': 'Produit', 'de_D...",6,[],...,None,None,None,None,None,None,None,None,None,None


--- categories --------------------------


...

,code,enabled
0,af_ZA,False
1,am_ET,False
2,ar_AE,False
3,ar_BH,False
4,ar_DZ,False
...,...,...
205,zh_HK,False
206,zh_MO,False
207,zh_SG,False
208,zh_TW,False


--- measurements --------------------------


,code,labels,standard_unit_code,units
0,Angle,"{'de_DE': 'Winkel', 'en_US': 'Angle', 'fr_FR':...",RADIAN,"{'RADIAN': {'code': 'RADIAN', 'labels': {'en_U..."
1,Area,"{'ca_ES': 'Àrea', 'da_DK': 'Areal', 'de_DE': '...",SQUARE_METER,{'SQUARE_MILLIMETER': {'code': 'SQUARE_MILLIME...
2,Binary,"{'ca_ES': 'Binari', 'da_DK': 'Binær', 'de_DE':...",BYTE,"{'CHAR': {'code': 'CHAR', 'labels': {'en_US': ..."
3,Brightness,"{'de_DE': 'Helligkeit', 'en_US': 'Brightness',...",LUMEN,"{'LUMEN': {'code': 'LUMEN', 'labels': {'de_DE'..."
4,Capacitance,"{'de_DE': 'Elektrische Kapazität', 'en_US': 'C...",FARAD,"{'ATTOFARAD': {'code': 'ATTOFARAD', 'labels': ..."
5,CaseBox,"{'ca_ES': 'Embalatge', 'da_DK': 'Emballage', '...",PIECE,"{'PIECE': {'code': 'PIECE', 'labels': {'ca_ES'..."
6,Decibel,"{'ca_ES': 'Decibel', 'da_DK': 'Decibel', 'de_D...",DECIBEL,"{'DECIBEL': {'code': 'DECIBEL', 'labels': {'ca..."
7,Duration,"{'ca_ES': 'Durada', 'da_DK': 'Varighed', 'de_D...",SECOND,"{'MILLISECOND': {'code': 'MILLISECOND', 'label..."
8,ElectricCharge,"{'ca_ES': 'Càrrega elèctrica', 'da_DK': 'Elekt...",AMPEREHOUR,{'MILLIAMPEREHOUR': {'code': 'MILLIAMPEREHOUR'...
9,Energy,"{'ca_ES': 'Energia', 'da_DK': 'Energi', 'de_DE...",JOULE,"{'JOULE': {'code': 'JOULE', 'labels': {'de_DE'..."


--- products --------------------------


,identifier,enabled,family,categories,groups,created,updated,parent,values
0,1111111171,True,accessories,"[master_accessories_bags, print_accessories, s...",[],2022-03-20 14:06:30+00:00,2022-03-20 14:06:30+00:00,None,"{'image': [{'locale': None, 'scope': None, 'da..."
1,13620748,True,led_tvs,"[led_tvs, samsung, tvs_projectors_sales]",[],2022-03-20 14:06:30+00:00,2022-03-20 14:06:30+00:00,None,"{'name': [{'locale': None, 'scope': None, 'dat..."
2,15554974,True,webcams,"[cameras_sales, logitech, webcams]",[],2022-03-20 14:06:30+00:00,2022-03-20 14:06:30+00:00,None,"{'maximum_video_resolution': [{'locale': None,..."
3,12249740,True,scanners,"[avision, print_scan_sales, scanners]",[],2022-03-20 14:06:52+00:00,2022-03-20 14:06:53+00:00,None,"{'picture': [{'locale': None, 'scope': None, '..."
4,1111111130,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],2022-03-20 14:06:55+00:00,2022-03-20 14:06:55+00:00,apollon_yellow,"{'size': [{'locale': None, 'scope': None, 'dat..."
...,...,...,...,...,...,...,...,...,...
1234,1111111125,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],2022-03-20 14:06:52+00:00,2022-03-20 14:06:53+00:00,apollon_pink,"{'size': [{'locale': None, 'scope': None, 'dat..."
1235,1111111126,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],2022-03-20 14:06:52+00:00,2022-03-20 14:06:53+00:00,apollon_red,"{'size': [{'locale': None, 'scope': None, 'dat..."
1236,1111111127,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],2022-03-20 14:06:52+00:00,2022-03-20 14:06:53+00:00,apollon_red,"{'size': [{'locale': None, 'scope': None, 'dat..."
1237,1111111128,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],2022-03-20 14:06:52+00:00,2022-03-20 14:06:53+00:00,apollon_yellow,"{'size': [{'locale': None, 'scope': None, 'dat..."


In [9]:
result = [
    {"__id__": prod.identifier, **prod.values}
    for prod in cache.products
]
print_and_safe_data("products__values", result)

--- products__values --------------------------


,__id__,image,ean,name,weight,display_diagonal,description,release_date,maximum_video_resolution,total_megapixels,...,power_requirements,eu_shoes_size,maximum_frame_rate,image_stabilizer,brand,material,meta_title,composition,care_instructions,main_color
0,1111111171,"[ProductValue(locale=None, scope=None, data='8...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='B...","[ProductValue(locale=None, scope=None, data={'...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13620748,NaN,NaN,"[ProductValue(locale=None, scope=None, data='S...",NaN,"[ProductValue(locale=None, scope=None, data={'...","[ProductValue(locale='en_US', scope='print', d...","[ProductValue(locale=None, scope='ecommerce', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15554974,NaN,NaN,"[ProductValue(locale=None, scope=None, data='L...",NaN,NaN,"[ProductValue(locale='en_US', scope='print', d...","[ProductValue(locale=None, scope='ecommerce', ...","[ProductValue(locale=None, scope=None, data='6...","[ProductValue(locale=None, scope=None, data='5')]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12249740,NaN,NaN,"[ProductValue(locale=None, scope=None, data='A...",NaN,NaN,"[ProductValue(locale='de_DE', scope='print', d...","[ProductValue(locale=None, scope='ecommerce', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1111111130,"[ProductValue(locale=None, scope=None, data='7...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='L...","[ProductValue(locale=None, scope=None, data={'...",NaN,"[ProductValue(locale='en_US', scope='ecommerce...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,1111111125,"[ProductValue(locale=None, scope=None, data='7...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='L...","[ProductValue(locale=None, scope=None, data={'...",NaN,"[ProductValue(locale='en_US', scope='ecommerce...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1235,1111111126,"[ProductValue(locale=None, scope=None, data='7...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='L...","[ProductValue(locale=None, scope=None, data={'...",NaN,"[ProductValue(locale='en_US', scope='ecommerce...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1236,1111111127,"[ProductValue(locale=None, scope=None, data='7...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='L...","[ProductValue(locale=None, scope=None, data={'...",NaN,"[ProductValue(locale='en_US', scope='ecommerce...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1237,1111111128,"[ProductValue(locale=None, scope=None, data='7...","[ProductValue(locale=None, scope=None, data='1...","[ProductValue(locale=None, scope=None, data='L...","[ProductValue(locale=None, scope=None, data={'...",NaN,"[ProductValue(locale='en_US', scope='ecommerce...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
